In [43]:
import tqdm
import random
import pathlib
import itertools
import collections
import re
import json
import shutil
import cv2
import numpy as np
import tensorflow as tf

import os
from os import listdir
from os.path import isfile, join
from pathlib import Path

In [44]:
path = "C:\CSP400\CSP400_ASLToText\Data\Videos\wlasl-complete\sample_classes"
files = [os.path.join(root, name)
             for root, dirs, files in os.walk(path)
             for name in files]

print(files)

['C:\\CSP400\\CSP400_ASLToText\\Data\\Videos\\wlasl-complete\\sample_classes\\a\\00295.mp4', 'C:\\CSP400\\CSP400_ASLToText\\Data\\Videos\\wlasl-complete\\sample_classes\\a\\01610.mp4', 'C:\\CSP400\\CSP400_ASLToText\\Data\\Videos\\wlasl-complete\\sample_classes\\a\\01611.mp4', 'C:\\CSP400\\CSP400_ASLToText\\Data\\Videos\\wlasl-complete\\sample_classes\\a\\01612.mp4', 'C:\\CSP400\\CSP400_ASLToText\\Data\\Videos\\wlasl-complete\\sample_classes\\a\\01613.mp4', 'C:\\CSP400\\CSP400_ASLToText\\Data\\Videos\\wlasl-complete\\sample_classes\\a\\01614.mp4', 'C:\\CSP400\\CSP400_ASLToText\\Data\\Videos\\wlasl-complete\\sample_classes\\a\\01615.mp4', 'C:\\CSP400\\CSP400_ASLToText\\Data\\Videos\\wlasl-complete\\sample_classes\\a\\66039.mp4', 'C:\\CSP400\\CSP400_ASLToText\\Data\\Videos\\wlasl-complete\\sample_classes\\a lot\\02123.mp4', 'C:\\CSP400\\CSP400_ASLToText\\Data\\Videos\\wlasl-complete\\sample_classes\\a lot\\02124.mp4', 'C:\\CSP400\\CSP400_ASLToText\\Data\\Videos\\wlasl-complete\\sample_cla

In [89]:
print("files[10]".split('\\')[-2])

IndexError: list index out of range

In [132]:
def Get_Class(filename):
    #print(filename)
    return filename.split("\\")[-2]

In [133]:
def Get_Class_Files(files):
    files_for_class = collections.defaultdict(list)
    for fname in files:
        class_name = Get_Class(fname)
        #print(class_name)
        files_for_class[class_name].append(fname)
    return files_for_class

In [134]:
def Select_Class_Subset(files_for_class, classes, files_per_class):
    files_subset = dict()

    for class_name in classes:
        class_files = files_for_class[class_name]
        files_subset[class_name] = class_files[:files_per_class]

    return files_subset

In [135]:
NUM_CLASSES = 10
FILES_PER_CLASS = 50

files_classes = Get_Class_Files(files)
classes = list(files_for_class.keys())

print('Num classes:', len(classes))
print('Num videos for class[0]:', len(files_for_class[classes[0]]))

Num classes: 10
Num videos for class[0]: 8


In [136]:
files_subset = Select_Class_Subset(files_for_class, classes[:NUM_CLASSES], FILES_PER_CLASS)
list(files_subset.keys())

['a',
 'a lot',
 'abdomen',
 'able',
 'about',
 'above',
 'accent',
 'accept',
 'accident',
 'accomplish']

In [137]:
def Split_Classes(files_for_class, count):
    split_files = []
    remainder = {}
    for cls in files_for_class:
        split_files.extend(files_for_class[cls][:count])
        remainder[cls] = files_for_class[cls][count:]
    return split_files, remainder

#def Split_Generator(files_for_class,splits):
    

In [147]:
files_for_class = Get_Class_Files(files)

classes = list(files_for_class.keys())[:NUM_CLASSES]

for cls in classes:
    random.shuffle(files_for_class[cls])

# Only use the number of classes you want in the dictionary
files_for_class = {x: files_for_class[x] for x in classes}

dirs = {}
splits = {"train": 30, "val": 10, "test": 10}
split_dir = pathlib.Path('../ASL_subset/')
for split_name, split_count in splits.items():
    print(split_name, ":")
    split_dir = split_dir / split_name
    split_files, files_for_class = Split_Classes(files_for_class, split_count)
    dirs[split_name] = split_dir

print(dirs)

train :
val :
test :
{'train': WindowsPath('../ASL_subset/train'), 'val': WindowsPath('../ASL_subset/train/val'), 'test': WindowsPath('../ASL_subset/train/val/test')}


In [146]:
video_count_train = len(list(split_dir.glob('train/*/*.avi')))
video_count_val = len(list(split_dir.glob('val/*/*.avi')))
video_count_test = len(list(split_dir.glob('test/*/*.avi')))
video_total = video_count_train + video_count_val + video_count_test
print(f"Total videos: {video_total}")

Total videos: 0
